Manufacturing AI Agent

- Install Dependencies
  
```bash 
pipenv shell
pipenv install langgraph langchain langchain-google-genai

```

In [ ]:
from langgraph.graph import StateGraph, END
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain.memory import ConversationBufferWindowMemory
from langchain_core.messages import HumanMessage

# --- LLM Setup ---
llm = ChatGoogleGenerativeAI(model="gemini-pro", temperature=0.3)

# --- Memory Setup ---
memory = ConversationBufferWindowMemory(k=5, return_messages=True)

# --- System Prompt ---
SYSTEM_PROMPT = """
You are a specialized control chart monitoring agent in a manufacturing environment.
Your job is to assess whether a process is trending out of control based on sensor readings and SPC rules.

Use this format:
@observe: Acknowledge the new reading.
@reason: Explain trend status or SPC rule violations.
@act: Recommend an action (e.g., monitor, alert supervisor, prep shutdown).
"""

# --- Sample Input ---
sensor_data = {
    "sensor": "vibration_motor_3A",
    "timestamp": "2025-06-18T11:47:09Z",
    "value": 0.89,
    "unit": "mm/s",
    "ucl": 0.90,
    "lcl": 0.50,
    "trend_window": [0.91, 0.94, 0.86, 0.89],
    "alert_enabled": True
}

# --- Define Nodes ---
def ingest(state):
    return {"sensor_data": state["input"]}

def analyze(state):
    formatted = f"{SYSTEM_PROMPT}\n\nNew telemetry data:\n{state['sensor_data']}"
    response = llm.invoke([HumanMessage(content=formatted)])
    memory.save_context({"input": state["sensor_data"]}, {"output": response.content})
    return {"response": response.content}

def output(state):
    print("\n===== AGENT RESPONSE =====")
    print(state["response"])
    return state

# --- Build Graph ---
graph = StateGraph()
graph.add_node("ingest", ingest)
graph.add_node("analyze", analyze)
graph.add_node("output", output)

graph.set_entry_point("ingest")
graph.add_edge("ingest", "analyze")
graph.add_edge("analyze", "output")
graph.add_edge("output", END)

# --- Compile and Run ---
app = graph.compile()
app.invoke({"input": sensor_data})
